In [0]:
import numpy as np

# Define global variables for the problem

p_h = 0.55
p_l = 0.45

prize = 1000
c_h = 50
c_l = 10

high = 0
low = 1

# number of actions
n_actions = 2
actions = [high, low]

# two outcomes
win = 0
lose = 1

n_outcomes = 2

# number of rounds
d = 3

# threshold
thresh = 4

# states encoded as 0, 1, ..., 2d
states = np.arange(0,2*d+1) 
n_states = len(states)

In [0]:
class Game:
    def __init__(self, init_state):
        self.initial_state = init_state
        self.state = self.initial_state
        self.reward = 0.0
        self.is_terminal = False

    def outcome(self, action):
        if action == high:
            win = np.random.binomial(1, p_h)
        else:
            win = np.random.binomial(1, p_l)
        return win

    def step(self, action):
        if action == high:
            self.reward = -c_h
        else:
            self.reward = -c_l    
      
        is_won = self.outcome(action)
        
        if self.state == 2*d-1 and is_won:
            self.state += 1
            self.reward += prize
            self.is_terminal = True
        elif self.state == 1 and not is_won:
            self.state -= 1
            self.is_terminal = True
        else:
            if is_won:
                self.state += 1
            else:
                self.state -= 1
            self.is_terminal = False
        return self.state, self.reward, self.is_terminal

    def reset(self):
        self.state = self.initial_state
        self.reward = 0.0
        self.is_terminal = False
        return self.state


def behavior_policy():
    return np.random.binomial(1, 0.5)


# Target policy is the threshold policy
def thresh_policy(state,action):
    if state < thresh:
      if action == high:
        return 0.8
      else:
        return 0.2
    else:
      if action == high:
        return 0.5
      else:
        return 0.5


def policy(x):
  pi_a = np.zeros((n_states,n_actions))
  for i in range(len(x)):
    pi_a[i+1, x[i]] = 1
  return pi_a

#target_policy = policy([low, high, high, low, low])

target_policy = policy([low, high, high, high, high])

In [0]:
init_state = d
env = Game(init_state)
episodes = 500000

q_values = np.zeros((n_states, n_actions))
c_values = np.zeros_like(q_values)

# No discount
gamma = 1.0


In [0]:
for i in range(1, episodes + 1):
  state = env.reset()
  history = []
  done = False
  t = 0

  while not done:
      a = behavior_policy()
      next_state, r, done = env.step(a)
        
      history.append([t, state, a, r])
      state = next_state
      t += 1

  g = 0.0
  w = 1.0

  for t, state, action, reward in history[::-1]:
      g = gamma * g + reward
      c_values[state, action] += w
      q_values[state, action] += w* (g - q_values[state, action]) / c_values[state, action]
      w *= target_policy[state, action] / 0.5
      if w == 0:
         break

In [0]:
print(q_values[1:2*d,:])

[[ -7.45533251  45.58610987]
 [155.37501915 108.87626375]
 [284.965277   295.2753474 ]
 [460.21471044 501.45294294]
 [706.16996782 784.92463592]]
